In [4]:
from mpify import ranch, in_torchddp
import os
import torch
import torch.distributed as dist

"""
    Applying mpify to the examples in PyTorch Distributed tutorial at
    https://pytorch.org/tutorials/intermediate/dist_tuto.html
"""

"""Blocking point-to-point communication."""
def run_blocking(rank, size):
    tensor = torch.zeros(1)
    if rank == 0:
        for r in range(1,size):
            tensor += 1
            # Send the tensor to process r
            dist.send(tensor=tensor, dst=r)
            print(f'Rank 0 started blocking send to rank {r}', flush=True)
    else:
        # Receive tensor from process 0
        dist.recv(tensor=tensor, src=0)
    print('Rank ', rank, ' has data ', tensor[0], flush=True)

"""Non-blocking point-to-point communication."""
def run_nonblocking(rank, size):
    print(f"Rank {rank}, name: {__name__}")
    tensor = torch.zeros(1)
    req = None
    if rank == 0:
        for r in range(1,size):
            tensor = tensor+1
            # Send the tensor to process r
            req = dist.isend(tensor=tensor, dst=r)
            print(f'Rank 0 started nonblocking send to rank {r}', flush=True)
            req.wait() # Must call req.wait() before next iteration, to avoid data corruption
    else:
        # Receive tensor from process 0
        req = dist.irecv(tensor=tensor, src=0)
        print(f'Rank {rank} started nonblocking receive', flush=True)
        req.wait()
    print('Rank ', rank, ' has data ', tensor[0], flush=True)

""" All-Reduce example."""
def run_allreduce(rank, size):
    """ Simple point-to-point communication. """
    tensor = torch.ones(1)
    dist.all_reduce(tensor, op=dist.ReduceOp.SUM)
    print('AllReduce: Rank ', rank, ' has data ', tensor[0], flush=True)

def rank_size_wrapper(fn):
    def new_fn(*args, **kwargs):
        import os
        return fn(int(os.environ.get('RANK')), int(os.environ.get('WORLD_SIZE')), *args, **kwargs)
    return new_fn


ModuleNotFoundError: No module named '__main__.mpify'; '__main__' is not a package

In [9]:
size = torch.cuda.device_count() if torch.cuda.is_available() else 5
    
imports='''
from mpify import in_torchddp, ddp_rank, ddp_worldsize
import os
import torch
import torch.distributed as dist
'''

in_torchddp(size, rank_size_wrapper(run_nonblocking), imports=imports)

Passing env to contextualize(): {}
Rank 0, name: __main__
Rank 0 started nonblocking send to rank 1
Rank 0 started nonblocking send to rank 2
Rank 0 started nonblocking send to rank 3
Rank 0 started nonblocking send to rank 4
Rank  0  has data  tensor(4.)
